In [1]:
# pip install dspy-ai
# https://medium.com/@frederick.ros/dspy-essentials-leveraging-assertions-for-optimized-data-handling-b9e8769cd087

In [8]:
import dspy
import random
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
my_name = dspy.Example(transcript="This is a transcript from a conversation",
                       action_items="and extracted action items").with_inputs("transcript")

In [10]:
# llm = dspy.OllamaLocal(model='gemma2', temperature=0.0)
llm = dspy.GROQ(model='gemma2-9b-it', api_key=os.getenv("GROQ_API_KEY"))
# llm = dspy.OpenAI(model='gpt-4o-mini', 
#                   max_tokens=4096, 
#                   temperature=1,
#                   )


In [11]:
dspy.settings.configure(lm=llm)

In [12]:
def varying_temp():
    temperature = 0.5 + 0.0001 * random.uniform(-1, 1)
    return {"config": {"temperature": temperature}}

In [13]:
class TranscriptExample(dspy.Signature):
    """Generate a short conversation transcript a discussion and extracted action_items"""
    transcript = dspy.OutputField(desc="A short transcript of a discussion (can be a monolog, like something you recording on your phone to not forget)")
    action_items = dspy.OutputField(desc="A comma-separated list of action items present in the transcript")

class GenerateExample(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_example = dspy.ChainOfThought(TranscriptExample)

    def forward(self):
        return self.generate_example(**varying_temp())


example_generator = GenerateExample()
example = example_generator()
example2 = example_generator()
example3 = example_generator()

print(example)
print(example2)
print(example3)

Prediction(
    rationale="Reasoning: Let's think step by step in order to generate a short conversation transcript about planning a weekend trip and extract action items from it. We'll imagine a conversation between two friends, Sarah and John, discussing their options.",
    transcript="Sarah: Hey John, are you free this weekend? I was thinking we could get out of the city for a bit. \n\nJohn: Yeah, I'm free! Where were you thinking of going? \n\nSarah: I don't know, something close by. Maybe that lake we talked about?  \n\nJohn:  The one with the hiking trails?  Sounds good to me.  Do you want to check the weather forecast?\n\nSarah:  Absolutely",
    action_items="Reasoning: Let's think step by step in order to generate a short conversation transcript about planning a weekend trip and extract action items from it. We'll imagine a conversation between two friends, Sarah and John, discussing their options. \n\nTranscript: Sarah: Hey John, are you free this weekend? I was thinking we 

In [48]:
import re

def check_format(action_items):
  """Check that the action items are a list of comma-separated action items"""
  # Simple check first: if we split according the , we should have more than 2 items
  if len(action_items.split(",")) == 1:
    return False
  
  # Now let's check it's not formated as 1.action item 1, blablah\n2. Action item 2, ..
  match = re.search(r'(\d)\.\s.+?(\\n|$)', action_items, re.MULTILINE)
  if match:
    return False
    
  return True

class GenerateExampleWithAssert(dspy.Module):
  def __init__(self):
    super().__init__()
    self.generate_example = dspy.ChainOfThought(TranscriptExample)
  
  def forward(self):
    ex = self.generate_example(**varying_temp())
    dspy.Assert(check_format(ex.action_items), "Action Items should be a comma-separated list")

    return ex

In [49]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

dspy.configure(trace=[])

generate_with_assert = assert_transform_module(GenerateExampleWithAssert(), backtrack_handler)

example = generate_with_assert()
example1 = generate_with_assert()
example2 = generate_with_assert()

print(example)
print(example1)
print(example2)


Prediction(
    rationale='produce the action_items. We need to identify the key points discussed in the conversation and extract any tasks or follow-ups mentioned.',
    transcript='Alice: Hey Bob, I was thinking we should finalize the project timeline by the end of this week. We need to make sure everyone is on the same page before we start the next phase. \nBob: Absolutely, I can draft a timeline and share it with the team for feedback. \nAlice: Great! Also, can you remind Sarah to send over the budget report? We need that for our meeting next Tuesday. \nBob: Sure thing. I’ll ping her about it. Anything else? \nAlice: Yes, let’s also set up a quick check-in with the marketing team to align on our messaging. We should do that by Thursday. \nBob: Got it. I’ll schedule that meeting.',
    action_items='finalize project timeline by end of the week, draft timeline and share with team for feedback, remind Sarah to send budget report, set up check-in with marketing team by Thursday, schedu

In [50]:
class ActionItemCompliance(dspy.Signature):
    """Check that all action items are included in the text"""
    text = dspy.InputField()
    action_items = dspy.InputField(desc="A comma-separated list of action items")
    comply : bool = dspy.OutputField(desc="True or False")

check_inclusion = dspy.TypedChainOfThought(ActionItemCompliance)

def are_action_items_included(transcript, action_items):
    comp = check_inclusion(text=transcript, action_items=action_items)
    return comp.comply

print(are_action_items_included(example.transcript, example.action_items))

True


In [51]:
class GenerateExampleWith2Assert(dspy.Module):
  def __init__(self):
    super().__init__()
    self.generate_example = dspy.ChainOfThought(TranscriptExample)
  
  def forward(self):
    ex = self.generate_example(**varying_temp())
    dspy.Assert(check_format(ex.action_items), "Action Items should be a comma-separated list")
    dspy.Assert(are_action_items_included(ex.transcript, ex.action_items), "Action Items should be included in the transcript")

    return ex
  
generate_with_assert = assert_transform_module(GenerateExampleWith2Assert(), backtrack_handler)

example = generate_with_assert()
print(example)

Prediction(
    rationale='produce the action_items. We need to identify the key points discussed, decisions made, and tasks assigned during the conversation.',
    transcript='Alice: "Hey team, we need to finalize the marketing strategy for the upcoming product launch. I think we should target social media influencers to reach a wider audience."\nBob: "That\'s a great idea, Alice. We should also consider creating some engaging video content to capture attention."\nCathy: "I can handle the video production if we set a timeline. How about we aim to have the first draft ready in two weeks?"\nAlice: "Perfect! Let\'s also make sure to allocate a budget for influencer partnerships. Bob, can you research potential influencers and their rates?"\nBob: "Absolutely, I\'ll have a list ready by next week."\nCathy: "And I’ll start working on the video concept as soon as we have the key messages defined."\nAlice: "Sounds good. Let\'s reconvene next Friday to review our progress."',
    action_items=

In [ ]:
# Seems like dspy is trying to fix this for non-OpenAI models.